<a href="https://colab.research.google.com/github/jdospina/viscomp202502/blob/main/notebooks/05_edge_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Detección de bordes

Notebook de demostración para detección de bordes. Ejecuta las celdas en Google Colab.


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

class ImageProcessor:
    """Clase para demostrar operaciones principales de procesamiento de imágenes."""

    def __init__(self):
        self.current_image = None

    def load_image(self, image_path: str | None = None, create_sample: bool = True):
        """Cargar una imagen desde disco o crear una imagen sintética.

        Parameters
        ----------
        image_path: str | None
            Ruta al archivo de imagen.
        create_sample: bool
            Si no se provee una ruta, genera una imagen sintética.
        """
        if image_path and os.path.exists(image_path):
            self.current_image = cv2.imread(image_path)
            self.current_image = cv2.cvtColor(self.current_image, cv2.COLOR_BGR2RGB)
            print(f"Imagen cargada: {image_path}")
        elif create_sample:
            self.current_image = self._create_sample_image()
            print("Imagen sintética creada para demostración")
        else:
            raise ValueError("No se pudo cargar la imagen")

        print(f"Dimensiones: {self.current_image.shape}")
        return self.current_image

    def _create_sample_image(self):
        """Crear una imagen sintética con patrones simples."""
        img = np.zeros((300, 400, 3), dtype=np.uint8)

        for i in range(300):
            img[i, :, 0] = int(255 * i / 300)  # Gradiente rojo

        img[50:150, 50:150, 2] = 255  # Rectángulo azul
        cv2.circle(img, (300, 100), 50, (0, 255, 0), -1)  # Círculo verde

        noise = np.random.randint(0, 50, img.shape, dtype=np.uint8)
        img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)
        return img

"""Detección de bordes y contornos."""




def run():
    """Demostrar detección de bordes y contornos."""
    print("=" * 60)
    print("5. DETECCIÓN DE BORDES Y CONTORNOS")
    print("=" * 60)

    processor = ImageProcessor()
    img = processor.load_image()
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    fig.suptitle("Detección de Bordes y Contornos", fontsize=16)

    axes[0, 0].imshow(img_gray, cmap="gray")
    axes[0, 0].set_title("Original")
    axes[0, 0].axis("off")

    edges_canny = cv2.Canny(img_gray, 50, 150)
    axes[0, 1].imshow(edges_canny, cmap="gray")
    axes[0, 1].set_title("Canny (50, 150)")
    axes[0, 1].axis("off")

    edges_canny2 = cv2.Canny(img_gray, 100, 200)
    axes[0, 2].imshow(edges_canny2, cmap="gray")
    axes[0, 2].set_title("Canny (100, 200)")
    axes[0, 2].axis("off")

    contours, hierarchy = cv2.findContours(edges_canny, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    img_contours = img.copy()
    cv2.drawContours(img_contours, contours, -1, (0, 255, 0), 2)
    axes[1, 0].imshow(img_contours)
    axes[1, 0].set_title(f"Contornos ({len(contours)} encontrados)")
    axes[1, 0].axis("off")

    lines = cv2.HoughLines(edges_canny, 1, np.pi / 180, threshold=100)
    img_lines = img.copy()
    if lines is not None:
        for rho, theta in lines[:10]:
            a = np.cos(theta)
            b = np.sin(theta)
            x0 = a * rho
            y0 = b * rho
            x1 = int(x0 + 1000 * (-b))
            y1 = int(y0 + 1000 * (a))
            x2 = int(x0 - 1000 * (-b))
            y2 = int(y0 - 1000 * (a))
            cv2.line(img_lines, (x1, y1), (x2, y2), (255, 0, 0), 2)

    axes[1, 1].imshow(img_lines)
    axes[1, 1].set_title("Detección de Líneas (Hough)")
    axes[1, 1].axis("off")

    circles = cv2.HoughCircles(img_gray, cv2.HOUGH_GRADIENT, 1, 20,
                              param1=50, param2=30, minRadius=0, maxRadius=0)

    img_circles = img.copy()
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for circle in circles[0, :]:
            cv2.circle(img_circles, (circle[0], circle[1]), circle[2], (255, 0, 0), 2)
            cv2.circle(img_circles, (circle[0], circle[1]), 2, (255, 0, 0), 3)

    axes[1, 2].imshow(img_circles)
    axes[1, 2].set_title("Detección de Círculos (Hough)")
    axes[1, 2].axis("off")

    plt.tight_layout()
    plt.show()


run()
